In [ ]:
# default_exp io.core

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import hashlib
import logging
import shutil
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, splitext

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

BUF_SIZE = 65536  # lets read stuff in 64kb chunks!
DEFAULT_IMAGE_EXTENSIONS = ['.jpg']
FOLDER_FILTER = ['.DS_Store']

In [ ]:
# export

logger = logging.getLogger(__name__)

# IO Core functions

> Helper functions for IO operations.

In [ ]:
# export


def scan_files(folder, file_extensions=None):
    """
    Scan the folder for files and filter files by file extension.
    If the optional `file_extension` is not set, **.jpg** as file extension is used by default.
    `folder`: the folder to scan for files
    `file_extensions`: the allowed file extensions
    return: the file path list
    """
    if file_extensions is None:
        file_extensions = DEFAULT_IMAGE_EXTENSIONS

    results = []
    if isdir(folder):
        files = listdir(folder)
        for file in files:
            file_path = join(folder, file)
            filename, file_extension = splitext(file_path)
            if isfile(file_path) and (file_extension in file_extensions):
                results.append(file_path)
            elif isdir(file_path):
                results.extend(scan_files(file_path, file_extensions))
    return results

In [ ]:
# export


def scan_path(path):
    """
    Scan a folder and sub-folders until the file level.
    `path`: the current directory to parse for folders
    return: a list of folders prefixed with base directory
    """
    has_sub_folders = False
    results = []
    names = listdir(path)
    for name in names:
        sub_folder_path = join(path, name)
        if isdir(sub_folder_path) and name not in FOLDER_FILTER:
            has_sub_folders = True
            results = results + scan_path(sub_folder_path)
    if not has_sub_folders:
        results.append(path)
    return results

In [ ]:
# export


def delete_folder(path):
    """
    Deletes a Folder, if exist.
    `path`: the folder path including the folder name
    return: the created folder path
    """
    if isdir(path):
        shutil.rmtree(path)
    return path

In [ ]:
# export


def create_folder(path, clear=False):
    """
    Creates a Folder, if not exist.
    `path`: the folder path including the folder name
    `clear`: whether or not the created folder should be empty
    return: the created folder path
    """

    if clear:
        # remove previous data-set folder, if exist to clear the contents
        delete_folder(path)

    if not isdir(path):
        makedirs(path)

    return path

In [ ]:
# export


def get_file_sha(file_path):
    """
    Calculates the SHA1 of a given file.
    `file_path`: the file path
    return: the calculated SHA1 as hex
    """
    result = None
    if isfile(file_path):
        sha1 = hashlib.sha1()
        with open(file_path, 'rb') as f:
            while True:
                data = f.read(BUF_SIZE)
                if not data:
                    break
                sha1.update(data)
            result = sha1.hexdigest()
    return result

In [ ]:
# export


def assign_filename_suffix(file_path, suffix=None):
    """
    Assign a filename suffix to a given file_path. (e.g <path>/filename_<suffix>.txt)
    `file_path`: the file path to suffix
    `suffix`: the suffix
    return: the suffixed file path
    """
    if suffix is None:
        return file_path
    path, ext = splitext(file_path)
    return "{}_{}{}".format(path, suffix, ext)

In [ ]:
# export


def strip_path(file_path, strip=os.sep):
    """
    Strip trailing separators from a path
    `file_path`: the file path to strip
    return: the striped file path
    """
    return file_path.rstrip(os.sep)


In [ ]:
# hide

# for generating scripts from notebook directly
from nbdev.export import notebook2script
notebook2script()

Converted annotation-core.ipynb.
Converted annotation-folder_category_adapter.ipynb.
Converted annotation-multi_category_adapter.ipynb.
Converted annotation-via_adapter.ipynb.
Converted annotation-yolo_adapter.ipynb.
Converted annotation_converter.ipynb.
Converted annotation_viewer.ipynb.
Converted category_tools.ipynb.
Converted core.ipynb.
Converted dataset-core.ipynb.
Converted dataset-image_classification.ipynb.
Converted dataset-image_object_detection.ipynb.
Converted dataset-image_segmentation.ipynb.
Converted dataset-type.ipynb.
Converted dataset_generator.ipynb.
Converted evaluation-core.ipynb.
Converted geometry.ipynb.
Converted image-color_palette.ipynb.
Converted image-inference.ipynb.
Converted image-opencv_tools.ipynb.
Converted image-pillow_tools.ipynb.
Converted image-tools.ipynb.
Converted index.ipynb.
Converted io-core.ipynb.
Converted tensorflow-tflite_converter.ipynb.
Converted tensorflow-tflite_metadata.ipynb.
Converted tensorflow-tfrecord_builder.ipynb.
Converted t